# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# make sure you visit WeatherPy --> output_data --> cities.csv
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Get rid of unnecessary extra index
del cities_df['Unnamed: 0']

cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cherskiy,68.7500,161.3000,-6.97,79,79,3.71,RU,1612592218
1,rikitea,-23.1203,-134.9692,77.58,75,23,16.49,PF,1612592219
2,port blair,11.6667,92.7500,83.03,64,0,12.10,IN,1612592219
3,punta arenas,-53.1500,-70.9167,44.60,65,40,8.05,CL,1612592220
4,tautira,-17.7333,-149.1500,80.60,69,40,3.44,PF,1612591631


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key) 

# Store latitude and longitude in locations, and humidity as humidity
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]

In [4]:
# Plot Heatmap
# center designates lat and lng center of map (had to do this because default settings were showing too much)
fig = gmaps.figure(center = (20,0), zoom_level=2.2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
Cloudiness = cities_df['Cloudiness']
max_temp = cities_df['Max Temp']
wind_speed = cities_df['Wind Speed']


ideal_df = cities_df.loc[(max_temp > 70) & (max_temp < 80)]
ideal_df = ideal_df.loc[(wind_speed < 10)]
ideal_df = ideal_df.loc[(Cloudiness == 0)]
ideal_df.dropna()
ideal_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
141,vao,-22.6667,167.4833,78.64,84,0,9.53,NC,1612592265
216,keetmanshoop,-26.5833,18.1333,73.40,40,0,4.61,NaN,1612592291
292,shima,24.4478,117.8111,77.00,53,0,6.71,CN,1612592309
361,nizwa,22.9333,57.5333,71.13,33,0,4.92,OM,1612592326
380,bandarbeyla,9.4942,50.8122,76.77,65,0,7.90,SO,1612592331
407,santarem,-2.4431,-54.7083,75.20,100,0,4.61,BR,1612592336
442,jodhpur,26.2867,73.0300,72.63,22,0,8.10,IN,1612592343
456,necochea,-38.5473,-58.7368,71.01,57,0,3.00,AR,1612592346
479,springbok,-29.6643,17.8865,73.40,23,0,3.44,ZA,1612592352
500,mandera,3.9366,41.8670,78.78,61,0,9.24,KE,1612592356


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# create hotel_df

temp_df = ideal_df.copy()
temp_df['Hotel Name'] = ""
temp_df['Hotel Address'] = ""
hotel_df = temp_df[['Hotel Name', 'Hotel Address','City', 'Country', 'Lat', 'Lng']]
hotel_df


,Hotel Name,Hotel Address,City,Country,Lat,Lng
141,,,vao,NC,-22.6667,167.4833
216,,,keetmanshoop,NaN,-26.5833,18.1333
292,,,shima,CN,24.4478,117.8111
361,,,nizwa,OM,22.9333,57.5333
380,,,bandarbeyla,SO,9.4942,50.8122
407,,,santarem,BR,-2.4431,-54.7083
442,,,jodhpur,IN,26.2867,73.0300
456,,,necochea,AR,-38.5473,-58.7368
479,,,springbok,ZA,-29.6643,17.8865
500,,,mandera,KE,3.9366,41.8670


In [7]:
# set the required params
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

# loop through each row of hotel df
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Use try and except to capture missing a data points
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\sthakur\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
# Drop all rows that had missing hotel results
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']
hotel_df

,Hotel Name,Hotel Address,City,Country,Lat,Lng
141,OURE TERA beach resort,Kuto,vao,NC,-22.6667,167.4833
216,Hotel Schützenhaus,Keetmanshoop,keetmanshoop,NaN,-26.5833,18.1333
292,雅斯特酒店,龙海紫崴路42号泷澄集团内,shima,CN,24.4478,117.8111
361,Tanuf Residency Hotel,Nizwa,nizwa,OM,22.9333,57.5333
380,Super secret pirate cove,Somalia,bandarbeyla,SO,9.4942,50.8122
407,Hotel Palace Santarém,"Av. Rui Barbosa, 726 - Santana, Santarém",santarem,BR,-2.4431,-54.7083
442,HOTEL RATAN VILAS,"Loco Shed Road, Ratanada, Jodhpur",jodhpur,IN,26.2867,73.0300
456,Hotel Ñikén Spa & Business Center,"Calle 87 335, Necochea",necochea,AR,-38.5473,-58.7368
479,Annie's Cottage - best b&b in Springbok,"King St, Springbok",springbok,ZA,-29.6643,17.8865
500,Granada hotel Mandera,"kenya, 482—70300 Mandera",mandera,KE,3.9366,41.8670


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))